In [1]:
# this script will generate all the results for the subfolders in the pretrained_results: cv2_results, pspnet_results, semantic_results
%load_ext autoreload
%autoreload 2


In [2]:
import torch
torch.cuda.is_available()


False

In [4]:
# %pip install -r requirements.txt

In [5]:
# %%python -m pip install torch

In [6]:
import os, sys
os.getcwd()
sys.path.append('/Users/yoon/Documents/gatech/spring24/WallAI')
sys.path.append('/Users/yoon/Documents/gatech/spring24/WallAI/semantic_segmentation_pytorch')



In [7]:

import torch, numpy, scipy.io, PIL.Image, torchvision.transforms
# Our libs


from semantic_segmentation_pytorch.mit_semseg.models import ModelBuilder, SegmentationModule
from semantic_segmentation_pytorch.mit_semseg.utils import colorEncode


def return_semantic_result(filename: str):
    # Network Builders
    net_encoder = ModelBuilder.build_encoder(
        arch='resnet50dilated',
        fc_dim=2048,
        weights='/Users/yoon/Documents/gatech/spring24/WallAI/semantic_segmentation_pytorch/ckpt/ade20k-resnet50dilated-ppm_deepsup/encoder_epoch_20.pth')
    net_decoder = ModelBuilder.build_decoder(
        arch='ppm_deepsup',
        fc_dim=2048,
        num_class=150,
        weights='/Users/yoon/Documents/gatech/spring24/WallAI/semantic_segmentation_pytorch/ckpt/ade20k-resnet50dilated-ppm_deepsup/decoder_epoch_20.pth',
        use_softmax=True)


    crit = torch.nn.NLLLoss(ignore_index=-1)
    segmentation_module = SegmentationModule(net_encoder, net_decoder, crit)
    segmentation_module.eval()
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    segmentation_module.to(DEVICE)
    # segmentation_module.cuda()

    # Load and normalize one image as a singleton tensor batch
    pil_to_tensor = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], # These are RGB mean+std values
            std=[0.229, 0.224, 0.225])  # across a large photo dataset.
    ])

    # pil_image = PIL.Image.open('our_data/Test/33012f501af95eecd41803710646e57e87c2680d.jpg').convert('RGB')
    pil_image = PIL.Image.open(filename).convert('RGB')
    img_original = numpy.array(pil_image)
    img_data = pil_to_tensor(pil_image)
    # singleton_batch = {'img_data': img_data[None].cuda()}
    singleton_batch = {'img_data': img_data[None].to(DEVICE)}
    output_size = img_data.shape[1:]


    # Run the segmentation at the highest resolution.
    with torch.no_grad():
        scores = segmentation_module(singleton_batch, segSize=output_size)

    # Get the predicted scores for each pixel
    _, pred = torch.max(scores, dim=1)
    pred = pred.cpu()[0].numpy()

    def visualize_wall(img, pred, class_to_display=0):
        """
            Function for visualizing wall prediction
            (original image, segmentation mask and original image with the segmented wall)
        """
        img_green = img.copy()
        black_green = img.copy()
        img_green[pred == class_to_display] = [111, 209, 201]
        black_green[pred == class_to_display] = [111, 209, 201]
        black_green[pred != class_to_display] = [0, 0, 0]
        # im_vis = numpy.concatenate((img, black_green, img_green), axis=1)
        im_vis = numpy.concatenate((img, img_green), axis=1)
        return PIL.Image.fromarray(im_vis)

        # display(PIL.Image.fromarray(im_vis))

    predicted_classes = numpy.bincount(pred.flatten()).argsort()[::-1]
    visualize_wall(img_original, pred, predicted_classes[0])
    
    # for c in predicted_classes[:15]:
    #     visualize_wall(img_original, pred, c)

img = return_semantic_result('/Users/yoon/Documents/gatech/spring24/WallAI/Data/Test/33a3ce25c7ce3f7cb7a14e0038a3dfff8f6a40a6.jpg')


/Users/yoon/miniforge3/envs/wall_ai/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/yoon/miniforge3/envs/wall_ai/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/yoon/miniforge3/envs/wall_ai/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <E459C462-F863-3A5A-AC9F-FD77B14BE845> /Users/yoon/miniforge3/envs/wall_ai/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Loading weights for net_encoder
Loading weights for net_decoder


In [8]:
# load the models

# semantic
import segmentation

hihih:  /Users/yoon/Documents/gatech/spring24/WallAI/pretrained_results
Loading weights for net_encoder
Loading weights for net_decoder
<class 'PIL.Image.Image'>


In [9]:

test_image_folder = '/Users/yoon/Documents/gatech/spring24/WallAI/Data/Test'
test_folder_to_save_to = '/Users/yoon/Documents/gatech/spring24/WallAI/pretrained_results/semantic_results'

# semantic
def process_images_in_folder(folder_path, folder_to_save_to):
    # Iterate through all files and directories in the specified folder
    count = 0
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # if count < 10:
            if True:
                # Construct the full path of the image file
                image_path = os.path.join(root, file)
                # Open the image using PIL
                # img = Image.open(image_path)
                
                # Perform any processing here if needed
                # img = segment_image(segmentation_module, image_path)
                img = segmentation.return_semantic_result(image_path)

                filename, extension = os.path.splitext(file)

                # Save the processed image to a file
                img.save(os.path.join(folder_to_save_to, filename + "_segmentation" + extension))  # Saving with a prefix "processed_"
                # print(f"Processed image: {file}")

                count += 1

process_images_in_folder(test_image_folder, test_folder_to_save_to)


hihih:  /Users/yoon/Documents/gatech/spring24/WallAI/pretrained_results
Loading weights for net_encoder
Loading weights for net_decoder
hihih:  /Users/yoon/Documents/gatech/spring24/WallAI/pretrained_results
Loading weights for net_encoder
Loading weights for net_decoder
hihih:  /Users/yoon/Documents/gatech/spring24/WallAI/pretrained_results
Loading weights for net_encoder
Loading weights for net_decoder
hihih:  /Users/yoon/Documents/gatech/spring24/WallAI/pretrained_results
Loading weights for net_encoder
Loading weights for net_decoder
hihih:  /Users/yoon/Documents/gatech/spring24/WallAI/pretrained_results
Loading weights for net_encoder
Loading weights for net_decoder
hihih:  /Users/yoon/Documents/gatech/spring24/WallAI/pretrained_results
Loading weights for net_encoder
Loading weights for net_decoder
hihih:  /Users/yoon/Documents/gatech/spring24/WallAI/pretrained_results
Loading weights for net_encoder
Loading weights for net_decoder
hihih:  /Users/yoon/Documents/gatech/spring24/Wa

In [ ]:
# directory to original test and train data

In [ ]:
# loop and save the results in corresponding folders